In [ ]:
import numpy as np
from pynwb import NWBFile, TimeSeries, NWBHDF5IO
from pynwb.base import Images
from pynwb.image import GrayscaleImage
from pynwb.device import Device
from pynwb.ophys import (
    TwoPhotonSeries, 
    OpticalChannel, ImageSeries, ImageSegmentation,
    Fluorescence, MotionCorrection, CorrectedImageStack
)

from datetime import datetime
from dateutil.tz import tzlocal

In [ ]:
nwbfile = NWBFile(
    'my first synthetic recording',
    'EXAMPLE_ID',
    datetime.now(tzlocal()),
    experimenter='Dr. Bilbo Baggins',
    lab='Bag End Laboratory',
    institution='University of Middle Earth at the Shire',
    experiment_description='I went on an adventure with thirteen dwarves '
                           'to reclaim vast treasures.',
    session_id='LONELYMTN'
)

device = nwbfile.create_device(
    name="Microscope",
    description="My two-photon microscope",
    manufacturer="The best microscope manufacturer"
)

optical_channel = OpticalChannel(
    name="OpticalChannel",
    description="an optical channel",
    emission_lambda=500.
)

imaging_plane = nwbfile.create_imaging_plane(
    name="ImagingPlane",
    optical_channel=optical_channel,
    imaging_rate=30.,
    description="a very interesting part of the brain",
    device=device,
    excitation_lambda=600.,
    indicator="GFP",
    location="V1",
    grid_spacing=[.01, .01],
    grid_spacing_unit="meters",
    origin_coords=[1., 2., 3.],
    origin_coords_unit="meters"
)

# using internal data. this data will be stored inside the NWB file
image_series1 = TwoPhotonSeries(
    name='TwoPhotonSeries1',
    data=np.ones((1000, 100, 100)),
    imaging_plane=imaging_plane,
    rate=1.0,
    unit='normalized amplitude'
)

In [ ]:
corrected = ImageSeries(
    name='corrected',  # this must be named "corrected"
    data=np.ones((1000, 100, 100)),
    unit='na',
    format='raw',
    starting_time=0.0,
    rate=1.0
)

xy_translation = TimeSeries(
    name='xy_translation',
    data=np.ones((1000, 2)),
    unit='pixels',
    starting_time=0.0,
    rate=1.0,
)

corrected_image_stack = CorrectedImageStack(
    corrected=corrected,
    original=image_series1,
    xy_translation=xy_translation,
)

motion_correction = MotionCorrection(
    corrected_image_stacks=[corrected_image_stack]
)

In [ ]:
with NWBHDF5IO('aa.nwb', 'w') as io:
    io.write(nwbfile)